In [ ]:
import pickle
import os
import cv2
import numpy as np
import matplotlib.image as img
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.svm import SVC

In [ ]:
path = '../archive/kaggle_data_for_face_masks'
cat = os.listdir(path)
labels = [0, 1]
label_dict = dict(zip(cat,labels))

In [ ]:
label_dict

{'without_mask': 0, 'with_mask': 1}

In [ ]:
haar_data = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [ ]:
#Data Preprocessing
img_size = 200
data = []
target_var = []
for c in cat:
    data_path = os.path.join(path,c)
    images = os.listdir(data_path)
    for img in images:
        image_path = os.path.join(data_path,img)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        faces = haar_data.detectMultiScale(image)
        
        if len(faces) == 0 :
            face = cv2.resize(image,(img_size,img_size))
            face = np.array(face).flatten()
            data.append(face)
            target_var.append(label_dict[c])
        else:
            for x,y,w,h in faces:
                face = image[y:y+h, x:x+w]
                face = cv2.resize(image,(img_size,img_size))
                face = np.array(face).flatten()
                data.append(face)
                target_var.append(label_dict[c])
                

In [ ]:
len(data)

8700

In [ ]:
len(target_var)

8700

In [ ]:
data = np.array(data)
target_var = np.array(target_var)

In [ ]:
#Train Test Split
train_X, test_X, train_Y, test_Y = train_test_split(data, target_var, test_size=0.20, stratify=target_var, random_state=32)

In [ ]:
#Training DecisionTreeClassifier Model
clf = DecisionTreeClassifier()
clf.fit(train_X,train_Y)
pred_Y = clf.predict(test_X)
pickle.dump(clf,open('DecisionTreeModel.sav','wb'))

In [ ]:
np.mean(test_Y==pred_Y)

0.8057471264367816

In [ ]:
#Training SVM Model
svm_model = SVC()
svm_model.fit(train_X,train_Y)
pred_Y = svm_model.predict(test_X)
pickle.dump(svm_model,open('SVMModel.sav','wb'))

In [ ]:
np.mean(test_Y==pred_Y)

0.8597701149425288

In [ ]:
#Training K nearest neighbours Model
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(train_X,train_Y)
pred_knn = knn.predict(test_X)
pickle.dump(knn,open('KNNModel.sav','wb'))

In [ ]:
print(np.mean(pred_knn==test_Y))

0.774712643678161


In [ ]:
#Training Random Forest Model
rf = RandomForestClassifier(random_state=0, criterion="entropy")
rf.fit(train_X, train_Y) 
pred_rf = rf.predict(test_X)
pickle.dump(rf,open('RandomForest.sav','wb'))

In [ ]:
print(np.mean(pred_rf==test_Y))

0.8827586206896552


In [ ]:
def TestModel(path, label, model):
    accuracy = 0
    images = os.listdir(path)
    faceData=[]
    test_y = []
    for img in images:
        img_path = os.path.join(path,img)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        faces = haar_data.detectMultiScale(image)
        for x,y,w,h in faces:
            face = image[y:y+h, x:x+w]
            face = cv2.resize(image,(img_size,img_size))
            face = np.array(face).flatten()
            faceData.append(face)
            test_y.append([label])
            
    pred_y = model.predict(faceData) 
    return np.mean(pred_y==test_y)

In [ ]:
path_masked = '../../../Assg1/Mask Dataset/with_mask'
path_unmasked = '../../../Assg1/Mask Dataset/without_mask'

In [ ]:
#Loadinbg the saved models
filename_dt = 'DecisionTreeModel.sav'
dt_model = pickle.load(open(filename_dt, 'rb'))

filename_svm = 'SVMModel.sav'
svm_model = pickle.load(open(filename_svm, 'rb'))

filename_knn = 'KNNModel.sav'
knn_model = pickle.load(open(filename_knn, 'rb'))

filename_rf = 'RandomForest.sav'
rf_model = pickle.load(open(filename_rf, 'rb'))

In [ ]:
accuracy = TestModel(path_masked,1, dt_model)
print(accuracy)

1.0


In [ ]:
accuracy = TestModel(path_unmasked, 0, dt_model)
print(accuracy)

0.9758064516129032


In [ ]:
accuracy = TestModel(path_masked,1, svm_model)
print(accuracy)

0.8888888888888888


In [ ]:
accuracy = TestModel(path_unmasked, 0, svm_model)
print(accuracy)

0.9879032258064516


In [ ]:
accuracy = TestModel(path_masked,1, knn_model)
print(accuracy)

0.7777777777777778


In [ ]:
accuracy = TestModel(path_unmasked, 0, knn_model)
print(accuracy)

0.9879032258064516


In [ ]:
accuracy = TestModel(path_masked,1, rf_model)
print(accuracy)

0.9629629629629629


In [ ]:
accuracy = TestModel(path_unmasked, 0, rf_model)
print(accuracy)

0.9919354838709677
